<a href="https://colab.research.google.com/github/zhestyatsky/mipt-opt-project/blob/dev/racing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/zhestyatsky/mipt-opt-project.git
import sys
sys.path.append('/content/mipt-opt-project/src')

Cloning into 'mipt-opt-project'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 260 (delta 54), reused 64 (delta 28), pack-reused 157
Receiving objects: 100% (260/260), 3.67 MiB | 17.64 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [4]:
import os
os.chdir("mipt-opt-project/src")
!git checkout dev
os.listdir()

Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


['natasha.py',
 'utils.py',
 'spider_boost_experiments.ipynb',
 'spider_boost.py',
 'spider_boost_experiments_colab.ipynb']

In [0]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import utils
from spider_boost import spider_boost
from natasha import natasha_15, natasha_2

In [6]:
train_dataset = dsets.MNIST(root='~/.pytorch/MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
train_dataset = utils.DatasetWrapper(train_dataset)

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
class CustomModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CustomModel, self).__init__()
        self.conv1 = torch.nn.Linear(input_dim, 28)
        self.conv2 = torch.nn.Linear(28, output_dim)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        outputs = torch.sigmoid(self.conv2(x))
        return outputs
  

In [0]:
loss = torch.nn.CrossEntropyLoss().cuda()
regularizer = utils.regularizer

In [0]:
batch_size = 100 #2048
learning_rate = 0.0001
n_epochs = 20

In [0]:
gd_model = CustomModel(784, 10)
sb_model = copy.deepcopy(gd_model)
n15_model = copy.deepcopy(gd_model)
n2_model = copy.deepcopy(gd_model)

if torch.cuda.is_available():
    gd_model = gd_model.cuda()
    sb_model = sb_model.cuda()
    n15_model = n15_model.cuda()
    n2_model = n2_model.cuda()

In [29]:
%%time
gd_total_loss = utils.gradient_descent(train_dataset, batch_size, gd_model, loss, regularizer, learning_rate, n_epochs)


epoch: 0


CPU times: user 31 s, sys: 599 ms, total: 31.6 s
Wall time: 32 s


In [0]:
%%time
sb_total_loss = spider_boost(train_dataset, batch_size, sb_model, loss, regularizer, learning_rate, n_epochs)

epoch: 0


CPU times: user 39.9 s, sys: 1.03 s, total: 41 s
Wall time: 41.3 s


In [0]:
%%time
n_15_total_loss = natasha_15(train_dataset, batch_size, n15_model, loss, regularizer, learning_rate, n_epochs, sigma=10)

In [0]:
%%time
n_2_total_loss = natasha_2(train_dataset, n2_model, loss, regularizer, learning_rate, n_epochs)

In [0]:
plt.plot(gd_total_loss, label = 'SGD')
plt.plot(sb_total_loss, label = 'SpiderBOOST')
plt.plot(n_15_total_loss, label = 'Natasha 1.5')
plt.plot(n_2_total_loss, label = 'Natasha 2')
plt.legend()
plt.semilogy()